In [1]:
# The purpose of this code is
# to get dataframe of cell_label, brain_section_label, average_correlation_score, 

In [81]:
import os
import pandas as pd
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import SimpleITK as sitk
import pathlib
from ast import literal_eval
from scipy import stats

from abc_atlas_access.abc_atlas_cache.abc_project_cache import AbcProjectCache

In [3]:
download_base = Path('C:/programming_data/abc_download_root')
abc_cache = AbcProjectCache.from_s3_cache(download_base)

# download_base = Path('../../data/abc_atlas') # Path to the already downloaded data or s3fs-fuse mount.
# abc_cache = AbcProjectCache.from_local_cache(download_base)

abc_cache.current_manifest

'releases/20240330/manifest.json'

In [4]:
##### generating dataframe of needs #####
cell = abc_cache.get_metadata_dataframe(directory='MERFISH-C57BL6J-638850', file_name='cell_metadata_with_cluster_annotation')
cell.rename(columns={'x': 'x_section',
                     'y': 'y_section',
                     'z': 'z_section'},
            inplace=True)
cell.set_index('cell_label', inplace=True)

# extract your needs from cell metadata
cell_extract = cell.loc[:, ['brain_section_label',
                            'cluster_alias',
                            'average_correlation_score',
                            'x_section',
                            'y_section',
                            'z_section']]

# reconstructed coordinates
reconstructed_coords = abc_cache.get_metadata_dataframe(
    directory='MERFISH-C57BL6J-638850-CCF',
    file_name='reconstructed_coordinates',
    dtype={"cell_label": str}
)
reconstructed_coords.rename(columns={'x': 'x_reconstructed',
                                     'y': 'y_reconstructed',
                                     'z': 'z_reconstructed'},
                            inplace=True)
reconstructed_coords.set_index('cell_label', inplace=True)
cell_joined = cell_extract.join(reconstructed_coords, how='inner')

# ccf coordinates
ccf_coords = abc_cache.get_metadata_dataframe(
    directory='MERFISH-C57BL6J-638850-CCF',
    file_name='ccf_coordinates',
    dtype={"cell_label": str}
)
ccf_coords.rename(columns={'x': 'x_ccf',
                           'y': 'y_ccf',
                           'z': 'z_ccf'},
                  inplace=True)
ccf_coords.drop(['parcellation_index'], axis=1, inplace=True)
ccf_coords.set_index('cell_label', inplace=True)
cell_joined = cell_joined.join(ccf_coords, how='inner')

# parcellation annotation
parcellation_annotation = abc_cache.get_metadata_dataframe(directory='Allen-CCF-2020',
                                                           file_name='parcellation_to_parcellation_term_membership_acronym')
parcellation_annotation.set_index('parcellation_index', inplace=True)
parcellation_annotation.columns = ['parcellation_%s'% x for x in  parcellation_annotation.columns]
parcellation_annotation = parcellation_annotation.loc[:, ['parcellation_division',
                                                          'parcellation_structure',
                                                          'parcellation_substructure']]
cell_joined = cell_joined.join(parcellation_annotation, on='parcellation_index')

cell_joined.head(5)

,brain_section_label,cluster_alias,average_correlation_score,x_section,y_section,z_section,x_reconstructed,y_reconstructed,z_reconstructed,parcellation_index,x_ccf,y_ccf,z_ccf,parcellation_division,parcellation_structure,parcellation_substructure
cell_label,,,,,,,,,,,,,,,,
1019171907102340387-1,C57BL6J-638850.37,1408,0.596276,7.226245,4.148963,6.6,7.255606,4.007680,6.6,1160,7.495417,2.445872,7.455066,HPF,DG,DG-po
1104095349101460194-1,C57BL6J-638850.26,4218,0.641180,5.064889,7.309543,4.2,5.036436,7.264429,4.2,564,9.227966,6.133693,5.225024,P,TRN,TRN
1017092617101450577,C57BL6J-638850.25,4218,0.763531,5.792921,8.189973,4.0,5.784270,8.007646,4.0,761,9.344912,6.989939,6.002664,P,P-unassigned,P-unassigned
1018093344101130233,C57BL6J-638850.13,4218,0.558073,3.195950,5.868655,2.4,3.161528,5.719814,2.4,718,10.977068,4.398568,3.305223,cbf,arb,arb
1019171912201610094,C57BL6J-638850.27,4218,0.591009,5.635732,7.995842,4.4,5.618763,7.847877,4.4,761,8.997138,6.798329,5.827197,P,P-unassigned,P-unassigned


In [5]:
##### filtering by substructure #####
struct = 'LH'
pred = (cell_joined['parcellation_substructure'] == struct)

# use copy() to avoid SettingWithCopyWarning error
filtered = cell_joined[pred].copy()
print("number of cells:", len(filtered))
filtered.head(5)

number of cells: 2599


,brain_section_label,cluster_alias,average_correlation_score,x_section,y_section,z_section,x_reconstructed,y_reconstructed,z_reconstructed,parcellation_index,x_ccf,y_ccf,z_ccf,parcellation_division,parcellation_structure,parcellation_substructure
cell_label,,,,,,,,,,,,,,,,
1017092617101800268,C57BL6J-638850.38,5283,0.582496,5.995759,4.715432,6.8,6.226971,4.699358,6.8,179,7.207646,3.195783,6.413064,TH,LH,LH
1017092617101780513-1,C57BL6J-638850.38,5283,0.555154,4.753934,5.018805,6.8,4.922270,4.845414,6.8,179,7.155061,3.329147,5.083562,TH,LH,LH
1017092617101800358-1,C57BL6J-638850.38,5283,0.516931,5.717248,4.819773,6.8,5.794362,4.844641,6.8,179,7.179035,3.349707,5.970946,TH,LH,LH
1019171906102380019,C57BL6J-638850.42,5283,0.542683,6.115045,4.638671,7.6,6.065471,4.932627,7.6,179,6.454495,3.432822,6.244846,TH,LH,LH
1017092617101570581,C57BL6J-638850.38,5283,0.550196,6.072982,4.476326,6.8,6.176254,4.516339,6.8,179,7.226975,2.989120,6.356838,TH,LH,LH


In [48]:
##### extracting near cells (old) #####
address = 'C:/programming_data/abc_atlas_files/'
example_cfos = pd.read_csv(address + 'example_cfos_coords.csv')

# function for judjing near cells
def is_cell_near(cx, cy, cz, xx, yy, zz) :
    dis = ((cx - xx)**2 + (cy - yy)**2 + (cz - zz)**2)**0.5
    # SET CUSTOM distance range
    if dis <= 0.5:
        return 1
    else:
        return 0

# using reconstructed coords
# initialize cfos column
filtered['cfos'] = 0
for i in example_cfos.itertuples():
    xx = i.x
    yy = i.y
    zz = i.z
    cfos_count = filtered.apply(lambda i : is_cell_near(i['x_reconstructed'], i['y_reconstructed'], i['z_reconstructed'], xx, yy, zz), axis = 1)
    
    all_zero = (cfos_count == 0).all()
    if all_zero:
        print('UNTRACKTED CELLS: Cell_index ' + str(i.cell_index) + '\tappears to be out of the structural range.')
    else:
        filtered['cfos'] = filtered['cfos'] + cfos_count
filtered.head(5)

UNTRACKTED CELLS: Cell_index 1	appears to be out of the structural range.
UNTRACKTED CELLS: Cell_index 2	appears to be out of the structural range.
UNTRACKTED CELLS: Cell_index 3	appears to be out of the structural range.
UNTRACKTED CELLS: Cell_index 4	appears to be out of the structural range.
UNTRACKTED CELLS: Cell_index 5	appears to be out of the structural range.
UNTRACKTED CELLS: Cell_index 6	appears to be out of the structural range.
UNTRACKTED CELLS: Cell_index 8	appears to be out of the structural range.
UNTRACKTED CELLS: Cell_index 9	appears to be out of the structural range.
UNTRACKTED CELLS: Cell_index 10	appears to be out of the structural range.
UNTRACKTED CELLS: Cell_index 11	appears to be out of the structural range.
UNTRACKTED CELLS: Cell_index 12	appears to be out of the structural range.
UNTRACKTED CELLS: Cell_index 13	appears to be out of the structural range.
UNTRACKTED CELLS: Cell_index 14	appears to be out of the structural range.
UNTRACKTED CELLS: Cell_index 15	a

,brain_section_label,cluster_alias,average_correlation_score,x_section,y_section,z_section,x_reconstructed,y_reconstructed,z_reconstructed,parcellation_index,x_ccf,y_ccf,z_ccf,parcellation_division,parcellation_structure,parcellation_substructure,cfos
cell_label,,,,,,,,,,,,,,,,,
1017092617101800268,C57BL6J-638850.38,5283,0.582496,5.995759,4.715432,6.8,6.226971,4.699358,6.8,179,7.207646,3.195783,6.413064,TH,LH,LH,2
1017092617101780513-1,C57BL6J-638850.38,5283,0.555154,4.753934,5.018805,6.8,4.922270,4.845414,6.8,179,7.155061,3.329147,5.083562,TH,LH,LH,1
1017092617101800358-1,C57BL6J-638850.38,5283,0.516931,5.717248,4.819773,6.8,5.794362,4.844641,6.8,179,7.179035,3.349707,5.970946,TH,LH,LH,0
1019171906102380019,C57BL6J-638850.42,5283,0.542683,6.115045,4.638671,7.6,6.065471,4.932627,7.6,179,6.454495,3.432822,6.244846,TH,LH,LH,1
1017092617101570581,C57BL6J-638850.38,5283,0.550196,6.072982,4.476326,6.8,6.176254,4.516339,6.8,179,7.226975,2.989120,6.356838,TH,LH,LH,2


In [75]:
##### extracting near cells (new) #####
address = 'C:/programming_data/abc_atlas_files/'
example_cfos = pd.read_csv(address + 'example_cfos_coords.csv')

# function for judjing near cells
def is_cell_near(cx, cy, cz, xx, yy, zz) :
    dis = ((cx - xx)**2 + (cy - yy)**2 + (cz - zz)**2)**0.5
    # SET CUSTOM distance range
    if dis <= 0.25:
        return True
    else:
        return False

cfos_o = pd.DataFrame(columns=['cluster_alias', 'number'], dtype=int)
cfos_x = pd.DataFrame(columns=['cluster_alias', 'number'], dtype=int)

# using reconstructed coords
for i in filtered.itertuples():
    cx = i.x_reconstructed
    cy = i.y_reconstructed
    cz = i.z_reconstructed

    for j in example_cfos.itertuples():
        all_zero = 1
        if is_cell_near(cx, cy, cz, j.x, j.y, j.z):
            all_zero = 0
            break;

    if all_zero:
        if i.cluster_alias in cfos_x['cluster_alias'].values:
            cfos_x.loc[cfos_x['cluster_alias'] == i.cluster_alias, 'number'] += 1
        else:
            new = pd.DataFrame({'cluster_alias': [i.cluster_alias], 'number': [1]})
            cfos_x = pd.concat([cfos_x, new], ignore_index = True)
    else:
        if i.cluster_alias in cfos_o['cluster_alias'].values:
            cfos_o.loc[cfos_o['cluster_alias'] == i.cluster_alias, 'number'] += 1
        else:
            new = pd.DataFrame({'cluster_alias': [i.cluster_alias], 'number': [1]})
            cfos_o = pd.concat([cfos_o, new], ignore_index = True)

    cfos_o = cfos_o.sort_values(by='cluster_alias')
    cfos_x = cfos_x.sort_values(by='cluster_alias')

print(cfos_o.head(5))
print(cfos_x.head(5))

   cluster_alias  number
0           2200       1
1           2898       7
2           2899      18
3           2900      36
4           2901      18
   cluster_alias  number
0           1745       2
1           1790       1
2           1794       1
3           2009       2
4           2200       1


In [77]:
##### Loading Cluster Expression Data (in tuple) #####
address = 'C:/programming_data/abc_atlas_files/'
example_tuple = pd.read_csv(address + 'sample_tuple.csv')

example_tuple.head(5)

,cluster_alias,Xkr4,Gm1992,Gm19938,Gm37381,Rp1,Sox17,Gm37587,Gm37323,Mrpl15,...,Gm28340,Pkhd1,4930486I03Rik,Gm28653,Il17a,Il17f,Mcm3,Gm28065,6720483E21Rik,Paqr8
0,1745,"(7.94741678237915, 2.6097095012664795, 100)","(0.9834359884262085, 2.562854051589966, 100)","(2.0493767261505127, 3.4055159091949463, 100)","(0.0, 0.0, 100)","(0.0, 0.0, 100)","(0.0, 0.0, 100)","(0.0, 0.0, 100)","(0.0, 0.0, 100)","(0.7968159317970276, 2.2856740951538086, 100)",...,"(0.0, 0.0, 100)","(0.07756791263818741, 0.7756791710853577, 100)","(0.0, 0.0, 100)","(0.0, 0.0, 100)","(0.0, 0.0, 100)","(0.0, 0.0, 100)","(0.0, 0.0, 100)","(0.0, 0.0, 100)","(0.0, 0.0, 100)","(0.30834904313087463, 1.5196444988250732, 100)"
1,1790,"(8.948236465454102, 1.6274256706237793, 176)","(1.1843513250350952, 2.582833766937256, 176)","(3.435621738433838, 3.601335048675537, 176)","(0.0, 0.0, 176)","(0.038037918508052826, 0.5046300292015076, 176)","(0.0, 0.0, 176)","(0.0, 0.0, 176)","(0.0, 0.0, 176)","(1.497515082359314, 2.7858469486236572, 176)",...,"(0.0, 0.0, 176)","(0.0, 0.0, 176)","(0.0, 0.0, 176)","(0.0, 0.0, 176)","(0.0, 0.0, 176)","(0.0, 0.0, 176)","(0.0, 0.0, 176)","(0.0, 0.0, 176)","(0.032670099288225174, 0.4334178566932678, 176)","(0.48299407958984375, 1.7194888591766357, 176)"
2,1794,"(9.588398933410645, 0.9946861863136292, 249)","(1.5004398822784424, 2.810119152069092, 249)","(4.507795810699463, 3.4766998291015625, 249)","(0.0, 0.0, 249)","(0.031238088384270668, 0.49292871356010437, 249)","(0.0, 0.0, 249)","(0.0, 0.0, 249)","(0.0, 0.0, 249)","(1.5896358489990234, 2.8142549991607666, 249)",...,"(0.0, 0.0, 249)","(0.051746100187301636, 0.5780520439147949, 249)","(0.0, 0.0, 249)","(0.0, 0.0, 249)","(0.0, 0.0, 249)","(0.0, 0.0, 249)","(0.0, 0.0, 249)","(0.0, 0.0, 249)","(0.13049781322479248, 0.9150307178497314, 249)","(0.39178523421287537, 1.553818941116333, 249)"
3,2200,"(8.257760047912598, 2.034111738204956, 61)","(1.3543646335601807, 2.631150007247925, 61)","(3.1314847469329834, 3.456773281097412, 61)","(0.0, 0.0, 61)","(0.0, 0.0, 61)","(0.0, 0.0, 61)","(0.0, 0.0, 61)","(0.0, 0.0, 61)","(1.6454371213912964, 2.920619010925293, 61)",...,"(0.0, 0.0, 61)","(0.0, 0.0, 61)","(0.0, 0.0, 61)","(0.0, 0.0, 61)","(0.0, 0.0, 61)","(0.0, 0.0, 61)","(0.0, 0.0, 61)","(0.0, 0.0, 61)","(0.0, 0.0, 61)","(0.7683536410331726, 2.1646363735198975, 61)"
4,2301,"(7.639716625213623, 0.9841918349266052, 3)","(2.0165441036224365, 3.4927570819854736, 3)","(2.1723618507385254, 3.762641191482544, 3)","(0.0, 0.0, 3)","(2.1723618507385254, 3.762641191482544, 3)","(0.0, 0.0, 3)","(0.0, 0.0, 3)","(0.0, 0.0, 3)","(0.0, 0.0, 3)",...,"(0.0, 0.0, 3)","(0.0, 0.0, 3)","(0.0, 0.0, 3)","(0.0, 0.0, 3)","(0.0, 0.0, 3)","(0.0, 0.0, 3)","(0.0, 0.0, 3)","(0.0, 0.0, 3)","(0.0, 0.0, 3)","(0.0, 0.0, 3)"


In [90]:
##### t-test #####
# for..
gene = 'Gm19938'

def safe_literal_eval(x):
    try:
        return literal_eval(x)
    except (ValueError, SyntaxError):
        return (np.nan, np.nan, np.nan)

def extract_single_gene_info(df_tuple, gene_name):
    gene_idx = df_tuple.columns.get_loc(gene_name)

    gene_df = pd.DataFrame({
        'cluster_alias': df_tuple['cluster_alias'],
        'mean': df_tuple[gene_name].apply(lambda x: safe_literal_eval(x)[0]),
        'stdev': df_tuple[gene_name].apply(lambda x: safe_literal_eval(x)[1]),
        'n': df_tuple[gene_name].apply(lambda x: safe_literal_eval(x)[2])
    })

    return gene_df

gene_df = extract_single_gene_info(example_tuple, gene)
gene_df.head(5)

,cluster_alias,mean,stdev,n
0,1745,2.049377,3.405516,100.0
1,1790,3.435622,3.601335,176.0
2,1794,4.507796,3.476700,249.0
3,2200,3.131485,3.456773,61.0
4,2301,2.172362,3.762641,3.0


In [91]:
# return weighted mean, STDEV, and total weight
def calculate_weighted_stats(gene_df, cfos_df):
    merged_df = pd.merge(gene_df, cfos_df, on='cluster_alias', how='inner')
    valid_rows = merged_df.dropna(subset=['mean', 'stdev'])

    if valid_rows.empty:
        return np.nan, np.nan

    total_weight = valid_rows['number'].sum()
    weighted_mean = (valid_rows['mean'] * valid_rows['number']).sum() / total_weight

    variance_numer = (valid_rows['number'] * (valid_rows['stdev'] ** 2 + (valid_rows['mean'] - weighted_mean) ** 2)).sum()
    weighted_variance = variance_numer / total_weight
    weighted_stdev = np.sqrt(weighted_variance)

    return weighted_mean, weighted_stdev, total_weight

mean_o, stdev_o, n_o = calculate_weighted_stats(gene_df, cfos_o)
mean_x, stdev_x, n_x = calculate_weighted_stats(gene_df, cfos_x)
fold_change = mean_o - mean_x

t_stat, p_value = stats.ttest_ind_from_stats(mean_x, stdev_x, n_x, mean_o, stdev_o, n_o, equal_var=False)

print("fold change(log cfos_o/cfos_x): ", fold_change)
print("t-statistic: ", t_stat)
print("p-value: ", p_value)

# significance level
alpha = 0.05
if p_value < alpha:
    print("difference O")
else:
    print("difference X")

fold change(log cfos_o/cfos_x):  -0.024770972146388237
t-statistic:  0.1808172257940528
p-value:  0.8565572705121831
difference X
